<a href="https://colab.research.google.com/github/ikhlaspalakkattu/ai4all6b/blob/main/ai4all_ml_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime

# File names and their corresponding ticker symbols
files = [
    ("AMZN_Historical_Data.csv", "AMZN"),
    ("FB_Historical_Data.csv", "FB"),
    ("GOOG_Historical_Data.csv", "GOOG"),
    ("GOOGL_Historical_Data.csv", "GOOGL"),
    ("NFLX_Historical_Data.csv", "NFLX"),
    ("AAPL_Historical_Data.csv", "AAPL")
]

# Date range for filtering
start_date = pd.to_datetime("2012-01-01")
end_date = pd.to_datetime("2023-01-01")  # inclusive up to this date

merged_df = pd.DataFrame()

for filename, ticker in files:
    # Read CSV, parse 'Date' as a datetime
    df = pd.read_csv(filename)

    # Make sure the Date column is correctly formatted
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Drop rows where Date couldn't be parsed
    df = df.dropna(subset=['Date'])

    # Filter rows by date range (inclusive, if you want strictly before 2023-01-01 change <= to <)
    mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
    df = df.loc[mask].copy()

    # Add Ticker column
    df['Ticker'] = ticker

    # Optional: Clean numeric columns (remove commas, convert to numbers)
    for col in ['Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']:
        if col in df.columns:
            if col == 'Vol.':
                # Typically 'Vol.' may have K/M/B suffixes. You may want to extend parsing if that's the case.
                df[col] = df[col].astype(str).str.replace(',', '')
            else:
                df[col] = df[col].astype(str).str.replace(',', '')
                df[col] = pd.to_numeric(df[col], errors='coerce')

    merged_df = pd.concat([merged_df, df], ignore_index=True)

# Save to csv
merged_df.to_csv("Combined_Historical_Data.csv", index=False)

print("Merged CSV saved as 'Combined_Historical_Data.csv'")


Merged CSV saved as 'Combined_Historical_Data.csv'


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Load sentiment and stock article datasets
sentiment_df = pd.read_csv("sentiment_labeled_dataset.csv")
articles_df = pd.read_csv("stock_data_articles.csv")
price_df = pd.read_csv("Combined_Historical_Data.csv")  # from previous merging step

# Preprocess datasets
sentiment_df["title"] = sentiment_df["title"].str.strip().str.lower()
articles_df["Title"] = articles_df["Title"].str.strip().str.lower()

# Merge news with sentiment using the 'title' field
merged_articles = pd.merge(
    articles_df, sentiment_df, left_on='Title', right_on='title', how='inner'
)

# Convert publish date to datetime and join to stock price (based on symbol and date)
merged_articles['Publishdate'] = pd.to_datetime(merged_articles['Publishdate'])
price_df['Date'] = pd.to_datetime(price_df['Date'])
final_df = pd.merge(
    merged_articles,
    price_df,
    left_on=['symbol', 'Publishdate'],
    right_on=['Ticker', 'Date'],
    how='inner'
)

# Create target: 1 if price increases tomorrow, 0 otherwise
final_df = final_df.sort_values(['symbol', 'Publishdate'])
final_df['Next_Price'] = final_df.groupby('symbol')['Price'].shift(-1)
final_df['Price_Change'] = final_df['Next_Price'] - final_df['Price']
final_df['Target'] = (final_df['Price_Change'] > 0).astype(int)

# Feature Engineering: use emotion scores, label, etc.
feature_cols = ['sad', 'joy', 'love', 'anger', 'fear', 'surprise']
X = final_df[feature_cols]
y = final_df['Target']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)

# Train a simple logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluation
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.67      1.00      0.80         4

    accuracy                           0.67         6
   macro avg       0.33      0.50      0.40         6
weighted avg       0.44      0.67      0.53         6



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Read the merged data (as explained previously)
# sentiment_df: sentiment dataset
# articles_df: articles dataset
# price_df: historical prices

# 1. Load the data
sentiment_df = pd.read_csv("sentiment_labeled_dataset.csv")
articles_df = pd.read_csv("stock_data_articles.csv")
price_df = pd.read_csv("Combined_Historical_Data.csv")

# 2. Standardize titles for merging
sentiment_df["title"] = sentiment_df["title"].str.strip().str.lower()
articles_df["Title"] = articles_df["Title"].str.strip().str.lower()

# 3. Merge news articles with sentiment data
merged_articles = pd.merge(
    articles_df, sentiment_df, left_on='Title', right_on='title', how='inner'
)

# 4. Merge with stock prices based on symbol and date
merged_articles['Publishdate'] = pd.to_datetime(merged_articles['Publishdate'])
price_df['Date'] = pd.to_datetime(price_df['Date'])
final_df = pd.merge(
    merged_articles,
    price_df,
    left_on=['symbol', 'Publishdate'],
    right_on=['Ticker', 'Date'],
    how='inner'
)

# 5. Feature engineering
final_df = final_df.sort_values(['symbol', 'Publishdate'])
final_df['Next_Price'] = final_df.groupby('symbol')['Price'].shift(-1)
final_df['Price_Change'] = final_df['Next_Price'] - final_df['Price']
final_df['Target'] = (final_df['Price_Change'] > 0).astype(int)

# Drop any rows with missing data
final_df = final_df.dropna(subset=['Target', 'sad', 'joy', 'love', 'anger', 'fear', 'surprise'])

# 6. Feature selection
features = ['sad', 'joy', 'love', 'anger', 'fear', 'surprise']
X = final_df[features]
y = final_df['Target']

# 7. Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# 8. Model training with XGBoost
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# 9. Predictions and evaluation
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.67      1.00      0.80         4

    accuracy                           0.67         6
   macro avg       0.33      0.50      0.40         6
weighted avg       0.44      0.67      0.53         6



/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [02:16:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being s

In [ ]:
# Install pmdarima if not installed


import sys
import subprocess

try:
    import pmdarima as pm
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pmdarima"])
    import pmdarima as pm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# --- Step 1: Load your datasets ---
price_df = pd.read_csv("Combined_Historical_Data.csv")
sentiment_df = pd.read_csv("sentiment_labeled_dataset.csv")
articles_df = pd.read_csv("stock_data_articles.csv")

# --- Step 2: Preprocessing titles and merging sentiment with articles ---
sentiment_df["title"] = sentiment_df["title"].str.strip().str.lower()
articles_df["Title"] = articles_df["Title"].str.strip().str.lower()

merged_articles = pd.merge(
    articles_df,
    sentiment_df,
    left_on='Title',
    right_on='title',
    how='inner'
)

merged_articles['Publishdate'] = pd.to_datetime(merged_articles['Publishdate'])
price_df['Date'] = pd.to_datetime(price_df['Date'])

# --- Step 3: Aggregate daily sentiment scores ---
sentiment_cols = ['sad', 'joy', 'love', 'anger', 'fear', 'surprise']
daily_sentiment = merged_articles.groupby('Publishdate')[sentiment_cols].mean().reset_index()

# --- Step 4: Select a stock ticker and merge price with sentiment ---
ticker = 'AAPL'  # Change this as needed
stock = price_df[price_df['Ticker'] == ticker][['Date', 'Price']].sort_values('Date')

# Merge prices with daily sentiment
data = pd.merge(
    stock,
    daily_sentiment,
    left_on='Date',
    right_on='Publishdate',
    how='left'
)
data = data.set_index('Date')

# --- Step 5: Reindex to continuous daily dates and fill missing ---
full_idx = pd.date_range(start=data.index.min(), end=data.index.max(), freq='D')
data = data.reindex(full_idx)
data.index.name = 'Date'

data[sentiment_cols] = data[sentiment_cols].fillna(0)
data['Price'] = data['Price'].ffill()

# --- Step 6: Compute log returns for stationarity ---
data['LogPrice'] = np.log(data['Price'])
data['LogReturn'] = data['LogPrice'].diff()
data = data.dropna(subset=['LogReturn'])

# Align exogenous variables with returns
exog = data[sentiment_cols]

# --- Step 7: Split into train/test ---
train_size = int(len(data)*0.9)
train_endog = data['LogReturn'][:train_size]
test_endog = data['LogReturn'][train_size:]
train_exog = exog[:train_size]
test_exog = exog[train_size:]

# --- Step 8: Use auto_arima to find best SARIMAX order ---
print("Finding optimal SARIMAX order by auto_arima, this may take a while...")
auto_model = pm.auto_arima(train_endog,
                           exogenous=train_exog,
                           seasonal=False,
                           stepwise=True,
                           suppress_warnings=True,
                           error_action='ignore',
                           trace=True,
                           max_p=5, max_q=5, max_d=2)

print(f"Best SARIMAX order found: {auto_model.order}")

# --- Step 9: Fit SARIMAX with best order ---
model = SARIMAX(train_endog,
                exog=train_exog,
                order=auto_model.order,
                enforce_stationarity=False,
                enforce_invertibility=False)

model_fit = model.fit(disp=False)

# --- Step 10: Forecast log returns on test set ---
start = train_size
end = len(data) - 1
pred_returns = model_fit.predict(start=start, end=end, exog=test_exog)

# --- Step 11: Invert log returns to price forecast ---
last_train_price = data['Price'].iloc[train_size - 1]
price_forecast = [last_train_price * np.exp(pred_returns.iloc[0])]
for r in pred_returns.iloc[1:]:
    price_forecast.append(price_forecast[-1] * np.exp(r))

forecast_index = data.index[train_size:]

# --- Step 12: Evaluation ---
rmse = np.sqrt(mean_squared_error(data['Price'][train_size:], price_forecast))
print(f"Test RMSE: {rmse:.4f}")

# --- Step 13: Plot actual vs forecasted prices ---
plt.figure(figsize=(14, 7))
plt.plot(data.index, data['Price'], label='Actual Price')
plt.plot(forecast_index, price_forecast, label='SARIMAX Forecast', color='green')
plt.axvline(data.index[train_size], color='red', linestyle='--', label='Train/Test Split')
plt.title(f"SARIMAX Forecast of {ticker} Stock Price Using Sentiment Exogenous Features")
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()





ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
!pip uninstall numpy pmdarima -y
!pip install numpy==1.24.4
!pip install --no-cache-dir pmdarima



Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pmdarima 2.0.4
Uninstalling pmdarima-2.0.4:
  Successfully uninstalled pmdarima-2.0.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 70.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
arviz 0.22.0 requires numpy>=1.26.0, but you have numpy 1.24.4 which is incompatible.
scipy 1.16.0 requires numpy<2.6,>=1.25.2, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, b

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 142.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 236.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.1 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.1 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.1 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.1 which is incompatible.
opencv-python 4.12.0.88